## Lab 10 - Clasificación de Perros y Gatos

Stefano Aragoni, Carol Arévalo, Luis Diego Santos

### Importar Librerías


In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
import cv2
import random
import tensorflow as tf
from PIL import Image
from skimage import color, filters, measure

## 1.1 Lectura del Dataset

### Lectura y Procesamiento de Imagenes

In [2]:
def is_image_corrupted(file_path):
    try:
        with Image.open(file_path) as img:
            img.verify()
        return False
    except:
        return True

In [3]:
def load_images(path, size=(150, 150)):
    images = []
    labels = []

    dirs = ["Cat", "Dog"]

    for dir in dirs:
        elements = os.listdir(path+'/'+dir)
        random.shuffle(elements)

        contador = 0

        for file in elements:
            if file.endswith('.jpg'):
                img_path = os.path.join(path+'/'+dir, file)
                label = 1 if 'Dog' == dir else 0

                corrupta = is_image_corrupted(img_path)

                if corrupta:
                    continue

                # Leer imagen a color
                img = cv2.imread(img_path)

                try:
                    # Redimensionar imagen
                    img = cv2.resize(img, size)

                    # Convertir imagen a escala de grises
                    gray_img = color.rgb2gray(img)

                    # Aplicar umbral adaptativo para binarizar imagen
                    binary_img = gray_img > filters.threshold_local(gray_img, block_size=35)

                    # Label the objects in the binary image
                    labeled_img = measure.label(binary_img)

                    # Find the properties of the labeled objects
                    properties = measure.regionprops(labeled_img)

                    # Find the object with the largest area
                    max_area = 0
                    max_area_index = 0
                    for i, prop in enumerate(properties):
                        if prop.area > max_area:
                            max_area = prop.area
                            max_area_index = i

                    # Extract the coordinates of the bounding box of the object
                    min_row, min_col, max_row, max_col = properties[max_area_index].bbox

                    # Crop the image to the bounding box of the object
                    img = img[min_row:max_row, min_col:max_col]

                    # expand the image to 150x150
                    img = cv2.resize(img, size)

                    # Girar imagen aleatoriamente
                    if random.random() < 0.5:
                        opt = random.randint(0, 3)

                        if opt == 0:
                            img = cv2.flip(img, 1)

                        elif opt == 1:
                            img = cv2.rotate(img, cv2.ROTATE_180)

                        elif opt == 2:
                            img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)

                        elif opt == 3:
                            img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)


                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    
                    # Normalizar valores de píxeles
                    img = img / 255.0

                    # # show image on popup
                    # cv2.imshow('image', img)
                    # cv2.waitKey(0)
                    # cv2.destroyAllWindows()

                    images.append(img)
                    labels.append(label)
                            
                    if contador == 5000:
                        break

                    if contador % 100 == 0:
                        print("Imagenes procesadas: ", contador)  

                    contador += 1

                except Exception as e:
                    pass

    return np.array(images), np.array(labels)

### Importar Imagenes y Split de Dataset

In [4]:
# Definir ruta del conjunto de datos
data_path = './PetImages'

# Cargar imágenes y etiquetas
images, labels = load_images(data_path)

# Comprobar tamaños de los conjuntos
print('\nNúmero de imágenes por clase:', np.bincount(labels))

Imagenes procesadas:  0
Imagenes procesadas:  100
Imagenes procesadas:  200
Imagenes procesadas:  300
Imagenes procesadas:  400
Imagenes procesadas:  500
Imagenes procesadas:  600
Imagenes procesadas:  700
Imagenes procesadas:  800
Imagenes procesadas:  900
Imagenes procesadas:  1000
Imagenes procesadas:  1100
Imagenes procesadas:  1200
Imagenes procesadas:  1300
Imagenes procesadas:  1400
Imagenes procesadas:  1500
Imagenes procesadas:  1600
Imagenes procesadas:  1700
Imagenes procesadas:  1800
Imagenes procesadas:  1900
Imagenes procesadas:  2000
Imagenes procesadas:  2100
Imagenes procesadas:  2200
Imagenes procesadas:  2300
Imagenes procesadas:  2400
Imagenes procesadas:  2500
Imagenes procesadas:  2600
Imagenes procesadas:  2700
Imagenes procesadas:  2800
Imagenes procesadas:  2900
Imagenes procesadas:  3000
Imagenes procesadas:  3100
Imagenes procesadas:  3200
Imagenes procesadas:  3300
Imagenes procesadas:  3400
Imagenes procesadas:  3500
Imagenes procesadas:  3600
Imagenes proc

Corrupt JPEG data: 399 extraneous bytes before marker 0xd9


Imagenes procesadas:  800
Imagenes procesadas:  900
Imagenes procesadas:  1000
Imagenes procesadas:  1100
Imagenes procesadas:  1200
Imagenes procesadas:  1300
Imagenes procesadas:  1400
Imagenes procesadas:  1500


Corrupt JPEG data: 226 extraneous bytes before marker 0xd9


Imagenes procesadas:  1600
Imagenes procesadas:  1700
Imagenes procesadas:  1800
Imagenes procesadas:  1900
Imagenes procesadas:  2000
Imagenes procesadas:  2100
Imagenes procesadas:  2200
Imagenes procesadas:  2300
Imagenes procesadas:  2400
Imagenes procesadas:  2500
Imagenes procesadas:  2600
Imagenes procesadas:  2700
Imagenes procesadas:  2800


Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9


Imagenes procesadas:  2900
Imagenes procesadas:  3000
Imagenes procesadas:  3100
Imagenes procesadas:  3200
Imagenes procesadas:  3300
Imagenes procesadas:  3400
Imagenes procesadas:  3500
Imagenes procesadas:  3600
Imagenes procesadas:  3700
Imagenes procesadas:  3800
Imagenes procesadas:  3900
Imagenes procesadas:  4000


Imagenes procesadas:  4100
Imagenes procesadas:  4200
Imagenes procesadas:  4300
Imagenes procesadas:  4400


Corrupt JPEG data: 2230 extraneous bytes before marker 0xd9


Imagenes procesadas:  4500
Imagenes procesadas:  4600
Imagenes procesadas:  4700
Imagenes procesadas:  4800
Imagenes procesadas:  4900

Número de imágenes por clase: [5001 5001]


In [5]:
# Dividir conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Dividir 20 de prueba en 10 de validación y 10 de prueba
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# Verificar tamaños de los conjuntos
print('\nTamaño del conjunto de entrenamiento:', X_train.shape)
print('Tamaño del conjunto de validación:', X_val.shape)
print('Tamaño del conjunto de prueba:', X_test.shape)


Tamaño del conjunto de entrenamiento: (8001, 150, 150)
Tamaño del conjunto de validación: (1001, 150, 150)
Tamaño del conjunto de prueba: (1000, 150, 150)


In [6]:
from keras.utils import to_categorical

y_train = to_categorical(y_train, 2)
y_val = to_categorical(y_val, 2)

## 1.2 Construccion del Modelo

In [30]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
          epochs=10,
          verbose=1,
          steps_per_epoch=400,
          validation_data=(X_val, y_val))

test_loss, test_acc = model.evaluate(X_val, y_val, verbose=1)
print('\nTest accuracy:', test_acc)

Epoch 1/12


ValueError: in user code:

    File "/Users/ls/miniconda/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/Users/ls/miniconda/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/ls/miniconda/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/Users/ls/miniconda/lib/python3.10/site-packages/keras/engine/training.py", line 1055, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/Users/ls/miniconda/lib/python3.10/site-packages/keras/engine/training.py", line 1149, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/Users/ls/miniconda/lib/python3.10/site-packages/keras/engine/compile_utils.py", line 577, in u